### Creating and Persisting an ML Model

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [ ]:
df.describe()
df

In [ ]:
df.info

Create a subset of features as an example.

In [ ]:
include = ['school', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [ ]:
df.info

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [ ]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [ ]:
df.describe()

Drop the G3 score

In [ ]:
include = ['school', 'schoolsup', 'higher','Mjob', 'Fjob', 'studytime', 'paid', 'failures', 'qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [ ]:
import sklearn
from sklearn import preprocessing, neighbors, svm
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split

dependent_variable = 'qual_student'

df = pd.get_dummies(df, drop_first=True)
y = df[dependent_variable]
X = df[df.columns.difference([dependent_variable])]

#splitting the train and test sets
X_train, X_test, y_train,y_test= train_test_split(X, y, test_size=0.3)

In [ ]:
rfc = rf(criterion='gini', 
        n_estimators=5750,
        max_depth=5,
        min_samples_split=6,
        min_samples_leaf=6,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [ ]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(rfc, 'app/handlers/model3.pkl')

In [ ]:
query_df = pd.DataFrame({'Fjob_health' : pd.Series(0), 
                         'Fjob_other' : pd.Series(1), 
                         'Fjob_services': pd.Series(0),
                         'Fjob_teacher': pd.Series(0),
                         'Mjob_health': pd.Series(0),
                         'Mjob_other': pd.Series(0),
                         'Mjob_services': pd.Series(0),
                         'Mjob_teacher': pd.Series(1),
                         'failures': pd.Series(5),
                         'higher_yes': pd.Series(1),
                         'paid_yes': pd.Series(1),
                         'school_MS': pd.Series(0),
                         'studytime': pd.Series(20)})
query_df = pd.get_dummies(query_df, drop_first=True).astype(float)

In [ ]:
pred = rfc.predict(query_df)

In [ ]:
pred

In [ ]:
type(pred)

In [ ]:
from sklearn.feature_selection import SelectFromModel

rfc = rf(criterion='gini', 
        n_estimators=15750,
        max_depth=25,
        min_samples_split=5,
        min_samples_leaf=5,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)


## Testing Features
df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G3', 'G2', 'G1'], inplace=True) 


# print(df.select_dtypes(exclude=["number","bool_"]).columns)
df = pd.get_dummies(df, columns = df.select_dtypes(exclude=["number","bool_"]).columns)
print(sum(df['qual_student']))

y = df['qual_student']
X = df[df.columns.difference(['qual_student'])]


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

sel = SelectFromModel(rfc)
sel.fit(X_train, y_train)

selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

print(selected_feat)


In [ ]:

col = df.columns.difference(selected_feat)

X = df[df.columns.difference(col)]


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

sel = SelectFromModel(rfc)
sel.fit(X_train, y_train)

selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

print(selected_feat)

In [ ]:
df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G3', 'G2', 'G1'], inplace=True) 

top_values = selected_feat

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=["number","bool_"]).columns)

y = df['qual_student']
X = df[df.columns.difference(['qual_student'])]
col = df.columns.difference(top_values)

X = df[df.columns.difference(col)]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

In [ ]:
df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G3', 'G2', 'G1'], inplace=True) 

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=["number","bool_"]).columns)

y = df['qual_student']
X = df[df.columns.difference(['qual_student'])]
print(X.columns)

rfc = rf(criterion='gini', 
        n_estimators=5750,
        max_depth=25,
        min_samples_split=5,
        min_samples_leaf=5,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

print(rfc.feature_importances_)
importances_df = pd.DataFrame({'variable':X_train.columns, 'importance': rfc.feature_importances_})
top_N = importances_df.sort_values(by=['importance'], ascending=False)
print(top_N)

In [ ]:
most_important_features =  ['Medu', 'health', 'absences', 'age', 'Walc']

df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G3', 'G2', 'G1'], inplace=True) 

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=["number","bool_"]).columns)

y = df['qual_student']
col = df.columns.difference(most_important_features)
X = df[df.columns.difference(col)]
print(X.columns)

rfc = rf(criterion='gini', 
        n_estimators=5750,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=5,
        max_features='auto',
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

rfc.fit(X_train, y_train)
print('****Results****')
print("Accuracy: {:.4%}".format(rfc.score(X_test, y_test)))

In [ ]:
from sklearn.model_selection import GridSearchCV

rfc2 = rf(criterion='gini', 
        n_estimators=7000,
        oob_score=True,
        random_state=42,
        n_jobs=-1,
        verbose=0)

df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G3', 'G2', 'G1'], inplace=True) 


y = df['qual_student']
col = df.columns.difference(['Fedu', 'Medu', 'Walc', 'absences', 'age', 'failures', 'freetime',
       'goout', 'health'])
X = df[df.columns.difference(col)]
X = pd.get_dummies(X, columns = X.select_dtypes(exclude=["number","bool_"]).columns)
print(X.columns)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

param_grid = { 
    'max_features': ['sqrt'],
    'max_depth' : [5, 7, 9, 12],
    'min_samples_split': [4,6,8],
    'min_samples_leaf': [1, 2, 3]
}

CV_rfc = GridSearchCV(estimator=rfc2, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
print (CV_rfc.best_params_)

In [ ]:
print("Accuracy: {:.4%}".format(CV_rfc.best_estimator_.score(X_test, y_test)))

In [ ]:
CV_rfc.best_estimator_.feature_importances_
importances_df = pd.DataFrame({'variable':X_train.columns, 'importance': CV_rfc.best_estimator_.feature_importances_})
top_N = importances_df.sort_values(by=['importance'], ascending=False)
print(top_N)

In [ ]:
df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.iloc[391]

In [ ]:
df.iloc[391]['absences']
df.iloc[391]['goout']
df.iloc[391]['qual_student']

In [ ]:
def add_query_val(row, column):
    return "&" + column + "=" + str(row[column])

def query_create(data):
    result = []
    for i in data.index:
        query = "predict?" + "Fedu=" + str(df.iloc[i]["Fedu"])
        query += add_query_val(df.iloc[i], "Medu")
        query += add_query_val(df.iloc[i], "Walc")
        query += add_query_val(df.iloc[i], "absences")
        query += add_query_val(df.iloc[i], "age")
        query += add_query_val(df.iloc[i], "failures")
        query += add_query_val(df.iloc[i], "freetime")
        query += add_query_val(df.iloc[i], "goout")
        query += add_query_val(df.iloc[i], "health")
        result.append(query)
    return result


In [ ]:
query_create(df[df["qual_student"] == 1])